#  Testing through documentation

26.3 doctest — Test interactive Python examples   
https://docs.python.org/3/library/doctest.html


<b>doctest</b> lets you <b>test</b> your code by running <b>examples embedded in the documentation</b> and verifying that they produce the expected results. 

It works by parsing the help text to find examples, running them, then comparing the output text against the expected value. 

Many developers find doctest <b>easier</b> than unittest because in its simplest form, there is no API to learn before using it.

However, as the examples become more complex <b>the lack of fixture management</b> can make writing doctest tests more <b>cumbersome</b> than using unittest.

## 1 Getting Started

<b>doctest</b> looks for lines 

* <b>beginning</b> with ：

   * **>>>** the interpreter prompt to find the beginning of a test case. 

* <b>ended</b> with 
      
    * a blank line</b>, 
      
    * the next interpreter prompt.

Here, <b>fun_multiply()</b> has two examples given in the module: ```doctest_simple.py```

In [66]:
%%file ./code/doctest/doctest_simple.py

def fun_multiply(a, b):
    """
    >>> fun_multiply(2, 3)
    6
    >>> fun_multiply('a', 3)
    'aaa'
    """
    return a * b

Overwriting ./code/doctest/doctest_simple.py


To run the tests, use

<b>doctest as the main program</b> via the <b>-m</b> option to the interpreter:

* python **-m doctest** *.py

In [6]:
!python -m doctest ./code/doctest/doctest_simple.py

Usually **no output** is produced .It means **all the examples worked**

Pass ** -v ** to the script, and **doctest** prints a detailed log of what it’s trying, and prints a summary at the end:

In [7]:
!python -m doctest -v ./code/doctest/doctest_simple.py

Trying:
    fun_multiply(2, 3)
Expecting:
    6
ok
Trying:
    fun_multiply('a', 3)
Expecting:
    'aaa'
ok
1 items had no tests:
    doctest_simple
1 items passed all tests:
   2 tests in doctest_simple.fun_multiply
2 tests in 2 items.
2 passed and 0 failed.
Test passed.


**Examples** cannot usually stand on their own as explanations of a function,

**doctest** also lets you keep the **surrounding text** you would normally include in the documentation. 

**Intervening text** is **ignored**, and can have any format as long as it does not look like a test case.



In [21]:
%%file ./code/doctest/doctest_simple_with_docs.py

def fun_multiply(a, b):
    """Returns a * b.

    Works with numbers:
    
    >>> fun_multiply(2, 3)
    6

    and strings:
    
    >>> fun_multiply('a', 3)
    'aaa'
    """
    return a * b

def fun_add(a, b):
    """Returns a + b.

    Works with numbers:
    
    >>> fun_add(2, 3)
    5
    
    and strings:
    
    >>> fun_add('1', '3')
    '13'
    """
    return a + b

Overwriting ./code/doctest/doctest_simple_with_docs.py


The surrounding text in the updated docstring 

* <b>useful to a human reader</b>,

* <b>ignored by doctest</b>, 

the results are the same.

In [22]:
!python -m doctest -v ./code/doctest/doctest_simple_with_docs.py

Trying:
    fun_add(2, 3)
Expecting:
    5
ok
Trying:
    fun_add('1', '3')
Expecting:
    '13'
ok
Trying:
    fun_multiply(2, 3)
Expecting:
    6
ok
Trying:
    fun_multiply('a', 3)
Expecting:
    'aaa'
ok
1 items had no tests:
    doctest_simple_with_docs
2 items passed all tests:
   2 tests in doctest_simple_with_docs.fun_add
   2 tests in doctest_simple_with_docs.fun_multiply
4 tests in 3 items.
4 passed and 0 failed.
Test passed.


# run doctest directly 

The simplest way to start using doctest  is to 

* **end** each module M with:

```python
if __name__ == "__main__":
   import doctest
   doctest.testmod()
```

**doctest** then examines docstrings in module M.

In [17]:
%%file  ./code/doctest/doctest_with_testmod.py

def fun_multiply(a, b):
    """Returns a * b.

    Works with numbers:
    
    >>> fun_multiply(2, 3)
    6

    and strings:
    
    >>> fun_multiply('a', 3)
    'aaa'
    """
    return a * b

def fun_add(a, b):
    """Returns a + b.

    Works with numbers:
    
    >>> fun_add(2, 3)
    5
    
    and strings:
    
    >>> fun_add('1', '3')
    '13'
    """
    return a + b

if __name__ == "__main__":
    import doctest
    doctest.testmod()

Overwriting ./code/doctest/doctest_with_testmod.py


In [20]:
!python ./code/doctest/doctest_with_testmod.py

In [18]:
!python ./code/doctest/doctest_with_testmod.py -v

Trying:
    fun_add(2, 3)
Expecting:
    5
ok
Trying:
    fun_add('1', '3')
Expecting:
    '13'
ok
Trying:
    fun_multiply(2, 3)
Expecting:
    6
ok
Trying:
    fun_multiply('a', 3)
Expecting:
    'aaa'
ok
1 items had no tests:
    __main__
2 items passed all tests:
   2 tests in __main__.fun_add
   2 tests in __main__.fun_multiply
4 tests in 3 items.
4 passed and 0 failed.
Test passed.


## DocTest of iapws.iapws97

https://github.com/jjgomera/iapws


In [ ]:
!pip install iapws

In debian you can find in **official repositories** in jessie. 

In ubuntu it's in official repositories from ubuntu saucy (13.10)

In [ ]:
!sudo apt install iapws

In [30]:
%%file ./code/doctest/doctest_iapws97.py

from iapws import iapws97

if __name__ == "__main__":
    import doctest
    doctest.testmod(iapws97)

Overwriting ./code/doctest/doctest_iapws97.py


In [31]:
!python ./code/doctest/doctest_iapws97.py -v

Trying:
    water=IAPWS97(T=170+273.15,x=0.5)
Expecting nothing
ok
Trying:
    water.Liquid.cp, water.Vapor.cp, water.Liquid.w, water.Vapor.w
Expecting:
    4.3695 2.5985 1418.3 498.78
**********************************************************************
File "C:\Python35\lib\site-packages\iapws\iapws97.py", line 4285, in iapws.iapws97.IAPWS97
Failed example:
    water.Liquid.cp, water.Vapor.cp, water.Liquid.w, water.Vapor.w
Expected:
    4.3695 2.5985 1418.3 498.78
Got:
    (4.369498593233083, 2.5985140134188485, 1418.2631646730363, 498.7751280146938)
Trying:
    water=IAPWS97(T=325+273.15,x=0.5)
Expecting nothing
ok
Trying:
    water.P, water.Liquid.v, water.Vapor.v, water.Liquid.h, water.Vapor.h
Expecting:
    12.0505 0.00152830 0.0141887 1493.37 2684.48
**********************************************************************
File "C:\Python35\lib\site-packages\iapws\iapws97.py", line 4288, in iapws.iapws97.IAPWS97
Failed example:
    water.P, water.Liquid.v, water.Vapor.v, water.Liq

    _Backward3x_v_PT(670,80,"a")
Expected:
    0.001503831359
Got:
    0.0015038313585404727
Trying:
    _Backward3x_v_PT(710,50,"b")
Expecting:
    0.002204728587
**********************************************************************
File "C:\Python35\lib\site-packages\iapws\iapws97.py", line 2978, in iapws.iapws97._Backward3x_v_PT
Failed example:
    _Backward3x_v_PT(710,50,"b")
Expected:
    0.002204728587
Got:
    0.0022047285870574838
Trying:
    _Backward3x_v_PT(750,80,"b")
Expecting:
    0.001973692940
**********************************************************************
File "C:\Python35\lib\site-packages\iapws\iapws97.py", line 2980, in iapws.iapws97._Backward3x_v_PT
Failed example:
    _Backward3x_v_PT(750,80,"b")
Expected:
    0.001973692940
Got:
    0.0019736929401211155
Trying:
    _Backward3x_v_PT(630,20,"c")
Expecting:
    0.001761696406
**********************************************************************
File "C:\Python35\lib\site-packages\iapws\iapws97.py", line 298

## Further Reading

doctest – Testing through documentation   

   https://pymotw.com/2/doctest/index.html